In [15]:
# Imports
import tensorflow as tf
import cPickle as pickle
from cd_dist import *
from Arrow3D import *
from scipy import spatial
import os
from mayavi import mlab
from mayavi.mlab import *
from scipy.spatial.distance import pdist, squareform
import cv2
import numpy as np



In [16]:
path_to_ellipsoid = '/home/heid/Documents/master/pc2mesh/points2mesh/utils/ellipsoid/'
path_point_cloud_data = '/home/heid/Documents/master/pc2mesh/point_cloud_data/big/'
path_mesh_output = '/home/heid/Documents/master/pc2mesh/points2mesh/manual_loss/figures/'
path_to_plots = '/home/heid/Documents/master/pc2mesh/points2mesh/manual_loss/figures/tmp/'
movie_folder = '/home/heid/Documents/master/pc2mesh/points2mesh/manual_loss/figures/'

In [17]:
## Loss functions for gradient descent calculations

def unit(tensor):
    return tf.nn.l2_normalize(tensor, axis=1)
# Define loss functions
def laplace_coord(pred, lape_idx, block_id):
    vertex = tf.concat([pred, tf.zeros([1,3])], 0)
    indices = lape_idx[block_id - 1][:, :8]
    weights = tf.cast(lape_idx[block_id - 1][:, -1], tf.float32)
    
    weights = tf.tile(tf.reshape(tf.reciprocal(weights), [-1, 1]), [1,3])
    laplace = tf.reduce_sum(tf.gather(vertex, indices), 1)
    laplace = tf.subtract(pred, tf.multiply(laplace, weights))
    return laplace

def laplace_loss(pred1, pred2, lape_idx, block_id=1):
    lap1 = laplace_coord(pred1, lape_idx, block_id)
    lap2 = laplace_coord(pred2, lape_idx, block_id)
    
    l_loss = tf.reduce_mean(tf.reduce_sum(tf.square(tf.subtract(lap1,lap2)),1)) * 1500
    
    move_loss = tf.reduce_mean(tf.reduce_sum(tf.square(tf.subtract(pred1, pred2)), 1)) * 100
    move_loss = tf.cond(tf.equal(block_id,1), lambda:0., lambda:move_loss)
    return l_loss + move_loss

def pseudo_chamfer_loss(pred, positions):
    gt_pt = positions
    dist1, idx1, dist2, idx2 = nn_distance(gt_pt, pred)
#     point_loss = (tf.reduce_mean(dist1) + 0.55 * tf.reduce_mean(dist2)) * 3000
    point_loss = (0.55 * tf.reduce_mean(dist1) + 1 * tf.reduce_mean(dist2)) * 3000
    return point_loss
    
def edge_length_loss(pred,edges,block_id=1):
    nod1 = tf.gather(pred, edges[block_id  - 1][:, 0], axis=0)
    nod2 = tf.gather(pred, edges[block_id  - 1][:, 1], axis=0)
    edge = tf.subtract(nod1 , nod2 )
    edge_length = tf.reduce_sum(tf.square(edge), 1)
    edge_loss = tf.reduce_mean(edge_length) * 300
#     print edge_loss.shape
    return edge_loss
    
def normal_cosine_loss(gt_pt,gt_nm,pred,edges, block_id = 1):
    nod1 = tf.gather(pred, edges[block_id  - 1][:, 0], axis=0)
    nod2 = tf.gather(pred, edges[block_id  - 1][:, 1], axis=0)
    edge = tf.subtract(nod1 , nod2 )
    
    dist1, idx1, dist2, idx2 = nn_distance(gt_pt, pred)
    normal = tf.gather(gt_nm, tf.squeeze(idx2,0))
    normal = tf.gather(normal, edges[block_id  - 1][:, 0])
    cosine = tf.abs(tf.reduce_sum(tf.multiply(unit(normal), unit(edge)), 1))
    normal_loss = tf.reduce_mean(cosine) * 0.5
    
    return normal_loss

def inverse_density_loss(pred):
    _,distances = python_knn_bruteforce(pred,8)
    distances = tf.convert_to_tensor(distances[0], dtype=tf.float32)
    max_d = tf.reduce_max(distances)
    point_loss = tf.reduce_mean(tf.subtract(max_d, distances))
    return point_loss


In [18]:
def unpool(inputs,pool_idx,block_id=1):
    block_pool_idx = pool_idx[block_id - 1]
    X = inputs
    add_feat = (1 / 2.0) * tf.reduce_sum(tf.gather(X, block_pool_idx), 1)
    outputs = tf.concat([X, add_feat], 0)
    return outputs

In [19]:
def convert_support_to_tensor(to_convert):
    indices = tf.convert_to_tensor(to_convert[0], dtype=tf.int64)
    values = tf.convert_to_tensor(to_convert[1], dtype=tf.float32)
    d_shape = tf.convert_to_tensor(to_convert[2], dtype=tf.int64)
    return tf.SparseTensor(indices, values=values, dense_shape=d_shape)

In [20]:
def save_model(vertices,output,pc,prefix,num=1):
    vert = np.hstack((np.full([vertices.shape[0], 1], 'v'), vertices))
    face = np.loadtxt(path_to_ellipsoid+'face'+str(num)+'.obj', dtype='|S32')
    mesh = np.vstack((vert,face))
    
    result_name=pc.replace(".txt", "_result_p"+str(num)+".obj")
    result_name = prefix+result_name
    path_to_mesh = os.path.join(output, result_name)

    np.savetxt(output, mesh, fmt='%s', delimiter=' ')
    print "Saved to {}".format(output)
    

In [49]:
def normalize_pc(points):
    mean_points = np.mean(points,axis=0)
    points = points - mean_points
    max_val = np.max(np.abs(points)) *3
    return points / max_val

In [50]:
def load_ellipsoid():
    # Load Ellipsoid
    placeholders = {}
    pkl = pickle.load(open(path_to_ellipsoid+'info_ellipsoid.dat', 'rb'))
    coord = normalize_pc(pkl[0])
    #coord = pkl[0]
    
    pool_idx = pkl[4]
    faces = pkl[5]
    lape_idx = pkl[7]
    edges = []
    for i in range(1, 4):
        adj = pkl[i][1]
        edges.append(adj[0])
    for i in range(0,3):
        lape_idx[i] = np.array(lape_idx[i])
        lape_idx[i][lape_idx[i] == -1 ] = np.max(lape_idx[i]) + 1
    neighbors = [[],[],[],]
    neighbors[0] = [ [] for i in range(156) ]
    neighbors[1] = [ [] for i in range(618) ]
    neighbors[2] = [ [] for i in range(2466) ]
    i = 0
    for e in edges:
        for p in e:
            if p[0] == p[1]:
                continue
            neighbors[i][p[0]].append(p[1])
        i = i + 1
    
    placeholders["coord"] = coord
    placeholders["tCoord"] = tf.convert_to_tensor(coord, dtype=tf.float32)
    placeholders["support1"] = [ convert_support_to_tensor(s) for s in pkl[1]]
    placeholders["support2"] = [ convert_support_to_tensor(s) for s in pkl[2]]
    placeholders["support3"] = [ convert_support_to_tensor(s) for s in pkl[3]]
    placeholders["faces"] = [tf.convert_to_tensor(f, dtype=tf.int32) for f in faces ]
    placeholders["edges"] = [tf.convert_to_tensor(e, dtype=tf.int32) for e in edges ]
    placeholders["lape_idx"] = [tf.convert_to_tensor(l, dtype=tf.int32) for l in lape_idx ]
    placeholders["pool_idx"] = [tf.convert_to_tensor(p, dtype=tf.int32) for p in pool_idx ]
    placeholders["neighbors"] = neighbors
    return placeholders

In [51]:
def load_gt():
    # Load Object to compare Ellipsoid to
    #path_to_object = '/home/heid/Documents/master/pc2mesh/point_cloud_data/big/airplane_0396.txt'
    path_to_object = path_point_cloud_data+'airplane_0616.txt'
    #path_to_object = '/home/heid/Documents/master/pc2mesh/point_cloud_data/big/sofa_0262.txt'
    airplane = np.loadtxt(path_to_object, delimiter=',', dtype =np.float32 )[0:1024]
    gt_pos = airplane[:,0:3]
    gt_norm = airplane[:,3:6]
    #mesh_loss(tCoord, gt_pos, gt_norm, edges)
    #pseudo_chamfer_loss(tCoord, gt_pos)
    return [gt_pos,gt_norm]

In [52]:
def create_mesh(vertices,block_id=1):
    vert = np.hstack((np.full([vertices.shape[0], 1], 'v'), vertices))
    face = np.loadtxt(path_to_ellipsoid+'face'+str(block_id)+'.obj', dtype='|S32')
    mesh = np.vstack((vert, face))
    
    result_name = path_mesh_output+"_"+str(block_id)+"_pred.obj"
    np.savetxt(result_name, mesh, fmt='%s', delimiter=' ')
    return result_name

In [53]:
def plot_points(gt_pos, temp_coord, gradients,name,save_figure,show_figure,iteration=0,block_id=1):
    # Plot with mayavi
#     mlab.figure(bgcolor=(0,0,0),size=(1280,720))

    num_v = [156,618,2466]
    mlab.figure(bgcolor=(1,1,1))
    
    mlab.points3d(gt_pos[:,0], gt_pos[:,1], gt_pos[:,2],color=(0,1,0),scale_factor=0.02)
    #mlab.points3d(coord[:,0], coord[:,1], coord[:,2],color=(1,0,0),scale_factor=0.01)
    #mlab.points3d(temp_coord[:,0], temp_coord[:,1], temp_coord[:,2],color=(1,0,0),scale_factor=0.09)
    
    mesh_path = create_mesh(temp_coord,block_id)
    mesh_txt = np.loadtxt(mesh_path, dtype='|S32')
    mesh_vertices = np.delete(mesh_txt[:num_v[block_id-1]],0,1)
    mesh_faces = np.delete(mesh_txt[num_v[block_id-1]:],0,1).astype(int)
    mesh_faces = mesh_faces - 1
    x = mesh_vertices[:,0].astype(float)
    y = mesh_vertices[:,1].astype(float)
    z = mesh_vertices[:,2].astype(float)

    f = mlab.gcf()
    m = mlab.triangular_mesh(x,y,z,mesh_faces,opacity=0.5)
    
    u = gradients[:,0]
    v = gradients[:,1]
    w = gradients[:,2]
    # Define quiver arrows
    #obj = mlab.quiver3d(x, y, z, u, v, w, line_width=5, scale_factor=1)
#     m.enable_contours = True
#     m.contour.number_of_contours = 35
#     m.actor.actor.rotate_y(iteration*2)
#     m.actor.actor.rotate_z(iteration*2)
#     m.actor.actor.rotate_z((360.0/50.0)*iteration)
#     camera = f.scene.camera
#     camera.yaw()
#     camera.pitch(iteration*0.1)
    if save_figure:
        mlab.savefig(path_to_plots+name+'.png',figure=f, magnification=1)
        mlab.close()
    if show_figure:
        mlab.show()



In [54]:
def generate_movie(name): 
    video_name = movie_folder+name+'.avi'
    images = [img for img in os.listdir(path_to_plots) if img.endswith(".png")]
    im_reverse = images[::-1]
    images.extend(im_reverse)
    frame = cv2.imread(os.path.join(path_to_plots, images[0]))
    height, width, layers = frame.shape
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    video = cv2.VideoWriter(video_name, fourcc, 30.0,frameSize=(width,height))

    for image in images:
        video.write(cv2.imread(os.path.join(path_to_plots, image)))

    cv2.destroyAllWindows()
    video.release()

In [55]:
init = tf.global_variables_initializer()
tf.set_random_seed(666)
sess = tf.Session()
sess.run(init)

In [56]:
gt_pos,gt_norm = load_gt()
placeholders = load_ellipsoid()
features = placeholders["tCoord"]

step_size = 3e-3

ellipsoid = placeholders["coord"]
temp_coord = placeholders["coord"]

#plot_points(gt_pos,temp_coord,"fig_"+str(0),True,False)
# for i in range(0,5):
num_v = [156,618,2466]
features = tf.convert_to_tensor(temp_coord, dtype=tf.float32)
prev = gt_pos
for block_id in range(1, 2):
    for i in range(1,2):
        #gradient_total = tf.gradients(mesh_loss(features,gt_pos,gt_norm, edges),features,aggregation_method=tf.AggregationMethod.ADD_N)
        
        gradient_chamfer = tf.gradients(pseudo_chamfer_loss(features,gt_pos),features) 
        gradient_laplace = tf.gradients(laplace_loss(ellipsoid, features,placeholders["lape_idx"],
                                                    block_id=block_id),features)
        gradient_normal = tf.gradients(
                           normal_cosine_loss(gt_pos,gt_norm,features,
                                            placeholders["edges"],block_id=block_id),
                            features)

        gradients_chamfer_156 = sess.run(gradient_chamfer)[0]
        gradients_laplace_156 = sess.run(gradient_laplace)[0]
        gradients_normal_156 = sess.run(gradient_normal)[0]
        res_indices = np.zeros([num_v[block_id - 1],3])
        
        for j in range(0,gradients_chamfer_156.shape[0]):
            n = placeholders["neighbors"][block_id-1][j] 
            res_indices[j] += gradients_chamfer_156[j]
            res_indices[n] += 0.2*gradients_chamfer_156[j]

        for j in range(0,gradients_laplace_156[0].shape[0]):
            res_indices[gradients_laplace_156[1][j]] += gradients_laplace_156[0][j]
        for k in range(0,gradients_normal_156[0].shape[0]):
            res_indices[gradients_normal_156[1][k]] += gradients_normal_156[0][k]

        prev_coord = temp_coord
        temp_coord = temp_coord - res_indices * step_size
        features = tf.convert_to_tensor(temp_coord, dtype=tf.float32)

        plot_points(gt_pos,prev_coord,res_indices*-1.0*step_size,"fig_"+str(block_id)+"_"+str(i),True,False,i,block_id)

    #plot_points(gt_pos,prev_coord,res_indices*-1.0*step_size,"fig_"+str(i),False,True,51)
    if block_id < 3:
        features= unpool(features, placeholders["pool_idx"], block_id)
    prev = temp_coord
    temp_coord = sess.run(features)


In [57]:
sess.run(tf.global_variables_initializer())

In [58]:
ret = sess.run(gradient_edge)[0]

NameError: name 'gradient_edge' is not defined

In [ ]:
generate_movie("plane")

In [57]:
def plot_arrow3d():
    # Plot with Arrow3D
    # Generate some example data

    ################################
    #plotting
    ################################    
    fig = plt.figure(figsize=(15,15))
    ax = fig.add_subplot(111, projection='3d')

    ax.plot(gt_pos[:,0], gt_pos[:,1], gt_pos[:,2], '.', markersize=5, color='g', alpha=0.5)
    ax.plot(coord[:,0], coord[:,1], coord[:,2], '.', markersize=3, color='b', alpha=0.9)
    ax.plot(temp_coord[:,0], temp_coord[:,1], temp_coord[:,2], '.', markersize=5, color='r', alpha=0.9)
    #x.plot(new_coord_edge[:,0], new_coord_edge[:,1], new_coord_edge[:,2], '.', markersize=5, color='r', alpha=0.9)
    '''
    for v in eig_vec:
    #ax.plot([mean_x,v[0]], [mean_y,v[1]], [mean_z,v[2]], color='red', alpha=0.8, lw=3)
    #I will replace this line with:
    a = Arrow3D([mean_x, v[0]], [mean_y, v[1]], 
                [mean_z, v[2]], mutation_scale=20, 
                lw=3, arrowstyle="-|>", color="r")
    ax.add_artist(a)
    '''
    ax.set_xlabel('x_values')
    ax.set_ylabel('y_values')
    ax.set_zlabel('z_values')

    #plt.title('Eigenvectors')

    plt.draw()
    plt.show()
